In [ ]:
from IPython.display import display, Image, HTML, clear_output
import time


import requests
from bs4 import BeautifulSoup
import json


print("Please enter your User_ID:")
participant_id = input()
print("Please enter your age:")
age = input()
print("Please enter your gender, please type 'female', 'male', or 'prefer not to say':")
gender = input()
clear_output (wait = False) 

list_of_messages = [
    "Welcome to the Maths Ability test",
    "This test involves 13 calculation questions",
    "Each part of the question will be shown separately",
    "Please type your answer at the end of each question",
    "You have three attempts for each question",
    "Good luck!"]

for message in list_of_messages:
    print(message)
    time.sleep(2) #seconds shown can be customised
    clear_output (wait = False) 
    
list_of_questions = {
    "Question_1": {"question": ["Question 1", "7", "+", "5", "="], "answer": "12"},
    "Question_2": {"question": ["Question 2", "24", "+", "15", "="], "answer": "39"},
    "Question_3": {"question": ["Question 3", "100", "-", "57", "="], "answer": "43"},
    "Question_4": {"question": ["Question 4", "79", "-", "34", "="], "answer": "45"},
    "Question_5": {"question": ["Question 5", "12", "x", "4", "="], "answer": "48"},
    "Question_6": {"question": ["Question 6", "11", "x", "11", "="], "answer": "121"},
    "Question_7": {"question": ["Question 7", "13", "x", "5", "="], "answer": "65"},
    "Question_8": {"question": ["Question 8", "41", "+", "69", "-", "20", "="], "answer": "90"},
    "Question_9": {"question": ["Question 9", "10", "x", "4", "+", "19"], "answer": "59"},
    "Question_10": {"question": ["Question 10", "7", "x", "8", "-", "33", "="], "answer": "23"},
    "Question_11": {"question": ["Question 11", "300", "-", "24", "+", "94", "="], "answer": "370"},
    "Question_12": {"question": ["Question 12", "75", "+", "57", "-", "61",  "="], "answer": "71"},
    "Question_13": {"question": ["Question 13", "49", "x", "2", "-", "12",  "="], "answer": "86"},
}

def maths_ability_test(questions):
    total_time_taken = 0
    counter_for_correct_answers = 0
    
    for each_question, part in questions.items():
        numbers_and_operations = part["question"]
        correct_answer = part["answer"]
        
        for each_number_or_operation in numbers_and_operations:
            style = "color: blue; font-size: 50px;"
            fancy_version = HTML(f"<span style='{style}'>{each_number_or_operation}</span>")
            display(fancy_version)
            time.sleep(1)
            clear_output(wait=True)
        
        start_time = time.time()
        attempts = 0
        
        while attempts < 3:
            user_answer = input("Enter answer here: ")
            print(f"You entered {user_answer}")
            
            if user_answer == correct_answer:
                print("Correct answer!")
                counter_for_correct_answers += 1
                break
            else:
                print("Incorrect answer :(")
                attempts += 1
                
        if attempts == 3:
            print("You've reached the maximum number of attempts.")
        
        end_time = time.time()
        time_taken = end_time - start_time
        total_time_taken += time_taken
        print(f"You took {time_taken:.2f} seconds for this question.\n")
        
        time.sleep(2)
        clear_output(wait=True)
        
        
    num_questions = len(questions)
    average_time_per_question = total_time_taken / num_questions

    print(f"Total time taken for all questions: {total_time_taken:.2f} seconds")
    print(f"Average time taken per question: {average_time_per_question:.2f} seconds")
    print(f"Total correct answers: {counter_for_correct_answers} out of {num_questions}")

    return total_time_taken, average_time_per_question, counter_for_correct_answers


#check
total_time, average_time, score_achieved = maths_ability_test(list_of_questions)


def send_to_google_form(data_dict, form_url):
    ''' Helper function to upload information to a corresponding google form 
        You are not expected to follow the code within this function!
    '''
    form_id = form_url[34:90]
    view_form_url = f'https://docs.google.com/forms/d/e/{form_id}/viewform'
    post_form_url = f'https://docs.google.com/forms/d/e/{form_id}/formResponse'

    page = requests.get(view_form_url)
    content = BeautifulSoup(page.content, "html.parser").find('script', type='text/javascript')
    content = content.text[27:-1]
    result = json.loads(content)[1][1]
    form_dict = {}
    
    loaded_all = True
    for item in result:
        if item[1] not in data_dict:
            print(f"Form item {item[1]} not found. Data not uploaded.")
            loaded_all = False
            return False
        form_dict[f'entry.{item[4][0][0]}'] = data_dict[item[1]]
    
    post_result = requests.post(post_form_url, data = form_dict)
    return post_result.ok

data_dict = {
        "User_ID": participant_id,
        "Age": age,
        "Gender": gender,
        "Average time taken to answer each question": average_time,
        "Total time taken to complete entire test": total_time,
        "Score achieved (out of 15)": score_achieved,
}

form_url = "https://docs.google.com/forms/d/e/1FAIpQLScDg9Aq4SUM6sbC7GT2uzUGoEVDw9v4WLqZZ6Gi2Ax2r1CBUg/viewform?usp=sf_link"

print("Please read:")
print("")
print("We wish to record your response data")
print("to an anonymised public data repository. ")
print("Your data will be used for educational teaching purposes")
print("practising data analysis and visualisation.")
print("")
print("Please type 'yes' in the box below if you consent to the upload.")
result = input()

if result.lower() == "yes":
    success = send_to_google_form(data_dict, form_url)
    if success:
        print("Thanks! Your data will be uploaded.")
    else:
        print("There was a problem uploading your data.")
else: 
    print("No problem! We hope you enjoyed the test :)")

#### 